In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
import warnings
warnings.filterwarnings("ignore")

In [2]:
import splitfolders
splitfolders.ratio("Data", output="output",
    seed=1337, ratio=(.8, .2), group_prefix=None, move=False) # default values

Copying files: 2033 files [00:04, 472.93 files/s]


In [5]:
learning_rate = 0.0001
buffer_size = 519
batch_size = 16
epochs = 500
img_size = 224
key = 18

In [6]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

training_set = train_datagen.flow_from_directory('output/train/',
                                                 target_size=(224,224),
                                                 batch_size=16)

test_set = test_datagen.flow_from_directory('output/val',
                                            target_size=(224,224),
                                            batch_size=16)

Found 1619 images belonging to 18 classes.
Found 413 images belonging to 18 classes.


In [7]:
def InceptionResNetV2_model():

    engine = tf.keras.applications.Xception(
        # Freezing the weights of the top layer in the InceptionResNetV2 pre-traiined model
        include_top = False,

        # Use Imagenet weights
        weights = 'imagenet',

        # Define input shape to 224x224x3
        input_shape = (img_size , img_size , 3),

    )

    x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(engine.output)
    x =Dropout(0.85)(x)
    out = tf.keras.layers.Dense(key, activation = 'softmax', name = 'dense_output')(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = engine.input, outputs = out)
    # Compile the model

    model.compile(
        # Set optimizer to Adam(0.0001)
        optimizer = tf.keras.optimizers.Adam(learning_rate= 1e-4),
        #optimizer= SGD(lr=3e-4, momentum=0.9),
        # Set loss to binary crossentropy
        #loss = tf.keras.losses.sparse_categorical_crossentropy,
        loss = tf.keras.losses.CategoricalCrossentropy(),
        # Set metrics to accuracy
        metrics = ['accuracy']
    )

    return model

In [8]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                                            patience=2,
                                                            verbose=2,
                                                            factor=0.9,
                                                            min_lr=0.00001)
reduce_lr =  keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.00001)

In [35]:
def train():
    time_start = time.time()
    
    model = InceptionResNetV2_model()
    
    model.summary()
    history = model.fit(training_set, epochs= epochs , verbose = 2, validation_data = test_set,
                       callbacks=[early_stopping, reduce_lr , learning_rate_reduction])
    
    

    model.save_weights('layer22-wskindiseases1.h5')
    model.save('layer22-skindiseases1.h5')
    
    print('Model saved.')
    
    time_end = time.time()
    print('Training Time:', time_end - time_start)
    print('\n')

    return history


In [36]:
def test():
    #test_labels = np.array(test_labels)

    from tensorflow import keras
    print('Testing:')
    mod = InceptionResNetV2_model()
    mod.load_weights('Layer22-wskindiseases1.h5')
    mod.evaluate(test_set)
    
    #prob = mod.predict(test_set)
    #predIdxs = np.argmax(prob, axis=1) 


    print('\n')
    #print(classification_report(test_set.labels, predIdxs,target_names = key, digits=5))
    return predIdxs, prob, mod 

In [43]:
if __name__ == "__main__":
    
    train_history = train() 
    predIdxs,prob,  model = test()
    

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

Epoch 1/500
102/102 - 52s - loss: 2.7930 - accuracy: 0.1322 - val_loss: 2.5182 - val_accuracy: 0.2228
Epoch 2/500
102/102 - 48s - loss: 2.3348 - accuracy: 0.3039 - val_loss: 2.1364 - val_accuracy: 0.3487
Epoch 3/500
102/102 - 48s - loss: 1.9290 - accuracy: 0.4027 - val_loss: 1.8434 - val_accuracy: 0.4262
Epoch 4/500
102/102 - 49s - loss: 1.5983 - accuracy: 0.5046 - val_loss: 1.6286 - val_accuracy: 0.5085
Epoch 5/500
102/102 - 48s - loss: 1.2543 - accuracy: 0.6208 - val_loss: 1.4553 - val_accuracy: 0.5593
Epoch 6/500
102/102 - 48s - loss: 1.0066 - accuracy: 0.6998 - val_loss: 1.3265 - val_accuracy: 0.6077
Epoch 7/500
102/102 - 48s - loss: 0.8176 - accuracy: 0.7566 - val_loss: 1.1832 - val_accuracy: 0.6852
Epoch 8/500
102/102 - 48s - loss: 0.6226 - accuracy: 0.8314 - val_loss: 1.0264 - val_accuracy: 0.7312
Epoch 9/500
102/102 - 48s - loss: 0.4835 - accuracy: 0.8660 - val_loss: 0.9293 - val_accuracy: 0.7579
Epoch 10/500
102/102 - 48s - loss: 0.3754 - accuracy: 0.8956 - val_loss: 0.9011 - 

C:\Users\laptop\AppData\Roaming\Python\Python38\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Model saved.
Training Time: 1850.8980255126953


Testing:
26/26 [==============================] - 4s 97ms/step - loss: 0.8253 - accuracy: 0.8136 0s - loss: 0.7




NameError: name 'predIdxs' is not defined

In [28]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
input_path = 'Data/'
train_data = image_dataset_from_directory(directory=input_path,
                                          batch_size=16,
                                          image_size=(224, 224))
key = train_data.class_names

Found 2049 files belonging to 18 classes.


In [29]:
def read_directory():
    data_filenames = []
    data_labels = []
    for i in range (len(key)):
        for filename in os.listdir(input_path + key[i]):
                data_filenames.append(input_path +key[i]+'/' + filename)
                data_labels.append(i)
  
    return  data_filenames, data_labels

In [30]:
def build_test_tfrecord(train_filenames, train_labels):  # Generate TFRecord of training set 
    with tf.io.TFRecordWriter(test_tfrecord)as writer:
        for filename, label in zip(train_filenames, train_labels):
            image = open(filename, 'rb').read()

            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),  # img > Bytes
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))  # label > Int
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

In [31]:
def _parse_example(example_string):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }

    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    feature_dict['image'] = tf.io.decode_png(feature_dict['image'], channels=3)
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [img_size, img_size]) / 255.0
    return feature_dict['image'], feature_dict['label']

In [32]:
def get_test_dataset(test_tfrecord):
    raw_test_dataset = tf.data.TFRecordDataset(test_tfrecord)
    test_dataset = raw_test_dataset.map(_parse_example)

    return test_dataset

In [33]:
def data_Preprocessing( test_dataset):
    test_dataset = test_dataset.batch(batch_size)
    test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return  test_dataset

In [34]:
def InceptionResNetV2_model():

    engine = tf.keras.applications.Xception(
        # Freezing the weights of the top layer in the InceptionResNetV2 pre-traiined model
        include_top = False,

        # Use Imagenet weights
        weights = 'imagenet',

        # Define input shape to 224x224x3
        input_shape = (img_size , img_size , 3),

    )

    x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(engine.output)
    x =Dropout(0.85)(x)
    out = tf.keras.layers.Dense(len(key), activation = 'softmax', name = 'dense_output')(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = engine.input, outputs = out)
    # Compile the model

    model.compile(
        # Set optimizer to Adam(0.0001)
        optimizer = tf.keras.optimizers.Adam(learning_rate= 3e-4),
        #optimizer= SGD(lr=0.0001, momentum=0.9),
        # Set loss to binary crossentropy
        loss = tf.keras.losses.sparse_categorical_crossentropy,

        # Set metrics to accuracy
        metrics = ['accuracy']
    )


    return model

In [35]:
def test(test_labels):
    test_labels = np.array(test_labels)

    from tensorflow import keras
    print('Testing:')
    mod = InceptionResNetV2_model()
    mod.load_weights('layer22-wskindiseases1.h5')
    mod.evaluate(test_dataset)
    
    prob = mod.predict(test_dataset)
    predIdxs = np.argmax(prob, axis=1) 


    print('\n')
    print(classification_report(test_labels, predIdxs,target_names = key, digits=5))
    return test_labels, predIdxs, prob, mod

In [36]:
if __name__ == "__main__":
    
    test_filenames, test_labels = read_directory()
    test_tfrecord = 'XRay_test.tfrecords'
    build_test_tfrecord(test_filenames, test_labels)


    test_dataset = get_test_dataset(test_tfrecord)



    test_dataset = data_Preprocessing(test_dataset) 

    test_labels, predIdxs,prob, Model = test(test_labels)


Testing:
129/129 [==============================] - 14s 101ms/step - loss: 0.1672 - accuracy: 0.9619.1628 - accuracy


                                               precision    recall  f1-score   support

                           Bullous Pemphigoid    0.96905   0.97837   0.97368       416
Bullous congenital ichthyosiform erythroderma    1.00000   0.90909   0.95238        44
                        Bullous drug eruption    1.00000   1.00000   1.00000        38
                             Bullous impetigo    0.94118   0.96000   0.95050        50
         Bullous systemic lupus erythematosus    0.97222   0.94595   0.95890        74
                     Dermatitis Herpetiformis    0.93413   0.98734   0.96000       158
                              Diabetic Bullae    0.97333   0.93590   0.95425        78
              Epidermolysis bullosa acquisita    0.98667   0.97368   0.98013        76
                              Grovers Disease    0.94030   0.98438   0.96183        64
          

In [37]:
def dermai_accuracy(prob, prob_Desicon):
    dermai = []
    for i in range(len(prob)):
        catogery =[]
        for j in range(len(prob[i])):
            if prob[i][j] > prob_Desicon:
                catogery.append(j)
        dermai.append(catogery)
    Positive = []
    for i in range(len(dermai)):
        if test_labels[i] in dermai[i]:
            Positive.append(1)
    accuracy = (len(Positive)/len(test_labels))*100  
    return  accuracy,dermai

In [38]:
accuracy,dermai = dermai_accuracy(prob, 0.10)

In [39]:
accuracy

97.07174231332357